### Import Packages

In [1]:
import pandas as pd
import math
from google.cloud import storage


### Read in data

In [3]:
client = storage.Client(project="uk-election-406413")
bucket = client.bucket("ukelectiondata")
files = bucket.list_blobs()
file_names = [file.name for file in files]

election_results = bucket.blob("preprocessed_final_df.csv")
election_results.download_to_filename('C:\\Users\\wg4618\\Documents\\LeWagon\\preprocessed_final_df.csv')


### Prepare data

In [79]:
elections_polling_df = pd.read_csv('C:\\Users\\wg4618\\Documents\\LeWagon\\preprocessed_final_df.csv')
elections_polling_df.drop(elections_polling_df.columns[0], axis=1, inplace=True)


In [105]:
elections_polling_df['con_pre_ge_adjusted'] = elections_polling_df['con_share_prev']/elections_polling_df['mean_con_share_ge']*elections_polling_df['con_pre_ge_poll']
elections_polling_df['lab_pre_ge_adjusted'] = elections_polling_df['lab_share_prev']/elections_polling_df['mean_lab_share_ge']*elections_polling_df['lab_pre_ge_poll']
elections_polling_df['lib_pre_ge_adjusted'] = elections_polling_df['lib_share_prev']/elections_polling_df['mean_lib_share_ge']*elections_polling_df['lib_pre_ge_poll']
elections_polling_df['oth_pre_ge_adjusted'] = elections_polling_df['oth_share_prev']/elections_polling_df['mean_oth_share_ge']*elections_polling_df['oth_pre_ge_poll']

elections_polling_df.columns


Index(['constituency_id', 'year', '0-9', '10-19', '20-29', '30-39', '40-49',
       '50-59', '60-69', '70-79', '80+', 'Home_owners', 'Private_renters',
       'Social_renters', 'Asian', 'Black', 'Mixed', 'Other', 'White',
       'country/region', 'total_votes', 'con_votes', 'lab_votes', 'lib_votes',
       'oth_votes', 'con_share', 'lab_share', 'lib_share', 'oth_share',
       'con_share_prev', 'lab_share_prev', 'lib_share_prev', 'oth_share_prev',
       'mean_con_share_ge', 'mean_lab_share_ge', 'mean_lib_share_ge',
       'mean_oth_share_ge', 'con_pre_ge_poll', 'lab_pre_ge_poll',
       'lib_pre_ge_poll', 'oth_pre_ge_poll', 'con_pre_ge_adjusted',
       'lab_pre_ge_adjusted', 'lib_pre_ge_adjusted', 'oth_pre_ge_adjusted'],
      dtype='object')

In [117]:
con_app = 0.25
lab_app = 0.4
lib_app = 0.2
oth_app = 0.12


In [124]:
melted_df = pd.melt(elections_polling_df, id_vars=['constituency_id', 'year', 'Private_renters', 'Social_renters', #'Home_owners',
                                   '0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', #'80+',
                                   'White', 'Asian', 'Black', 'Mixed', #'Other',
                                   'con_pre_ge_adjusted', 'lab_pre_ge_adjusted', 'lib_pre_ge_adjusted', 'oth_pre_ge_adjusted',
                                   'con_share_prev','mean_con_share_ge', 'lab_share_prev','mean_lab_share_ge', 'lib_share_prev','mean_lib_share_ge', 'oth_share_prev', 'mean_oth_share_ge'],
                    value_vars=['con_votes', 'lab_votes', 'lib_votes', 'oth_votes'],
                    var_name='party', value_name='votes')

columns_to_drop = ["year", "constituency_id", "votes", 'con_share_prev','mean_con_share_ge', 'lab_share_prev','mean_lab_share_ge', 'lib_share_prev','mean_lib_share_ge', 'oth_share_prev', 'mean_oth_share_ge']
X_old = melted_df[melted_df['year'] != 2024].drop(columns=columns_to_drop)

X_new = melted_df[melted_df['year'] == 2024]
X_new['con_pre_ge_adjusted'] = X_new['con_share_prev']/X_new['mean_con_share_ge']*con_app
X_new['lab_pre_ge_adjusted'] = X_new['lab_share_prev']/X_new['mean_lab_share_ge']*lab_app
X_new['lib_pre_ge_adjusted'] = X_new['lib_share_prev']/X_new['mean_lib_share_ge']*lib_app
X_new['oth_pre_ge_adjusted'] = X_new['oth_share_prev']/X_new['mean_oth_share_ge']*oth_app

X_new.drop(columns=columns_to_drop, inplace=True)

y_old = melted_df[["votes"]][melted_df['year'] != 2024]


/tmp/ipykernel_20603/997114661.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new['con_pre_ge_adjusted'] = X_new['con_share_prev']/X_new['mean_con_share_ge']*con_app
/tmp/ipykernel_20603/997114661.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new['lab_pre_ge_adjusted'] = X_new['lab_share_prev']/X_new['mean_lab_share_ge']*lab_app
/tmp/ipykernel_20603/997114661.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,Private_renters,Social_renters,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,White,Asian,Black,Mixed,con_pre_ge_adjusted,lab_pre_ge_adjusted,lib_pre_ge_adjusted,oth_pre_ge_adjusted,party
4,0.162862,0.20449,0.1028,0.1242,0.1302,0.133,0.1188,0.1294,0.1236,0.0934,0.949201,0.026383,0.005276,0.01461,0.120476,0.655538,0.062712,0.196155,con_votes


### Encode the data

In [103]:
column_to_encode = 'party'
one_hot_encoded = pd.get_dummies(X_old[column_to_encode], prefix=column_to_encode)
X_old_encoded = pd.concat([X_old, one_hot_encoded], axis=1)
X_old_encoded = X_old_encoded.drop(column_to_encode, axis=1)

one_hot_encoded2 = pd.get_dummies(X_new[column_to_encode], prefix=column_to_encode)
X_new_encoded = pd.concat([X_new, one_hot_encoded2], axis=1)
X_new_encoded = X_new_encoded.drop(column_to_encode, axis=1)


### Sort out adjusted columns in new data

In [100]:


X_new_encoded['con_pre_ge_adjusted'] = X_new_encoded['con_share_prev']/X_new_encoded['mean_con_share_ge']*con_slider
X_new_encoded['lab_pre_ge_adjusted'] = X_new_encoded['lab_share_prev']/X_new_encoded['mean_lab_share_ge']*lab_slider
X_new_encoded['lib_pre_ge_adjusted'] = X_new_encoded['lib_share_prev']/X_new_encoded['mean_lib_share_ge']*lib_slider
X_new_encoded['oth_pre_ge_adjusted'] = X_new_encoded['oth_share_prev']/X_new_encoded['mean_oth_share_ge']*oth_slider


KeyError: 'con_share_prev'

### Split the data

In [85]:
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_old, y_old, test_size=0.2, random_state=42)


,Private_renters,Social_renters,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,White,Asian,Black,Mixed,con_pre_ge_adjusted,lab_pre_ge_adjusted,lib_pre_ge_adjusted,oth_pre_ge_adjusted,party
4402,0.158666,0.110214,0.1221,0.1120,0.0924,0.1280,0.1453,0.1431,0.1074,0.0916,0.921600,0.038991,0.008908,0.023339,0.593379,0.204108,0.217600,0.056366,lab_votes
8115,0.158738,0.133556,0.0927,0.1006,0.0863,0.0890,0.1144,0.1514,0.1537,0.1279,0.964637,0.012604,0.005542,0.014148,0.624935,0.210943,0.087199,0.106752,lib_votes
6491,0.159070,0.196018,0.1295,0.1159,0.1228,0.1159,0.1347,0.1419,0.1146,0.0788,0.973391,0.009352,0.005154,0.009795,0.203739,0.525416,0.060927,0.310613,lib_votes
5017,0.133246,0.083106,0.1154,0.1149,0.1044,0.1192,0.1265,0.1424,0.1099,0.0956,0.784438,0.155364,0.014334,0.026533,0.605893,0.178799,0.269001,0.045731,lab_votes
87,0.154156,0.107026,0.1059,0.1066,0.1108,0.1058,0.1289,0.1527,0.1300,0.1048,0.977375,0.010891,0.002272,0.006980,0.487666,0.495716,0.027376,0.071996,con_votes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7175,0.140338,0.118172,0.1077,0.1044,0.0880,0.1079,0.1376,0.1572,0.1292,0.1028,0.958554,0.019178,0.004966,0.013093,0.632160,0.266023,0.078073,0.084409,lib_votes
6496,0.185972,0.146948,0.1136,0.1109,0.1074,0.1067,0.1321,0.1435,0.1366,0.0924,0.958356,0.020132,0.005164,0.011355,0.426881,0.372123,0.064608,0.122757,lib_votes
6745,0.281154,0.380210,0.1546,0.1012,0.1852,0.2125,0.1379,0.0935,0.0584,0.0344,0.565332,0.141075,0.205147,0.059504,0.237695,0.546521,0.074421,0.150656,lib_votes
1076,0.127967,0.236987,0.1280,0.1122,0.1277,0.1244,0.1354,0.1397,0.1173,0.0723,0.974101,0.008977,0.002818,0.011261,0.195977,0.640941,0.056429,0.154376,con_votes


In [33]:
from sklearn.decomposition import PCA
pca = PCA
X_old
#pca.fit(X_old)
# Plot cumulative explained variance
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
plt.plot(cumulative_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Components')
plt.grid(True)
plt.show()


,Private_renters,Social_renters,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,White,Asian,Black,Mixed,con_pre_ge_adjusted,lab_pre_ge_adjusted,lib_pre_ge_adjusted,oth_pre_ge_adjusted,votes
0,0.109669,0.203430,0.1068,0.1242,0.1227,0.1246,0.1446,0.1311,0.1149,0.0778,0.977918,0.011438,0.002786,0.006774,0.118590,0.461759,0.172566,0.131377,4411.0
1,0.128666,0.203809,0.1128,0.1095,0.1232,0.1293,0.1289,0.1422,0.1209,0.0812,0.967662,0.016775,0.003675,0.009573,0.137766,0.576513,0.066652,0.327352,3742.0
2,0.136265,0.203960,0.1133,0.1126,0.1194,0.1337,0.1242,0.1410,0.1201,0.0836,0.963560,0.018910,0.004031,0.010692,0.143290,0.573062,0.044613,0.172790,5901.0
3,0.143864,0.204112,0.1118,0.1152,0.1187,0.1330,0.1228,0.1389,0.1191,0.0879,0.959457,0.021045,0.004387,0.011811,0.184584,0.555990,0.030841,0.130811,6518.0
5,0.163604,0.121358,0.0960,0.1131,0.0953,0.0998,0.1460,0.1319,0.1438,0.1017,0.978926,0.008572,0.001653,0.008190,0.324378,0.285521,0.250096,0.123166,10734.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11401,0.278386,0.188737,0.0960,0.1053,0.2575,0.1431,0.1090,0.1100,0.0813,0.0598,0.912297,0.047417,0.008335,0.020405,0.314939,0.532313,0.080529,0.000000,4277.0
11403,0.085029,0.082232,0.0989,0.1259,0.0982,0.1074,0.1523,0.1324,0.1321,0.0929,0.961311,0.023910,0.003415,0.009005,0.601088,0.187782,0.298865,0.000000,2056.0
11404,0.096706,0.084759,0.1003,0.1229,0.1139,0.0993,0.1358,0.1369,0.1282,0.0973,0.955380,0.024862,0.003952,0.011637,0.417179,0.189949,0.147616,0.072538,7809.0
11405,0.101377,0.085770,0.0994,0.1286,0.1107,0.0987,0.1271,0.1384,0.1223,0.1052,0.953008,0.025244,0.004166,0.012690,0.591220,0.290633,0.117617,0.071996,1094.0


In [ ]:
pca = PCA(n_components=8)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
X_new_pca = pca.transform(X_new)
